In [80]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u265-b01-0ubuntu2~18.04).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [81]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [82]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

3- Se tiene un RDD con información de vuelos programados con la forma (número de vuelo, código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM). A su vez, se cuenta con el registro actualizado del estado de los vuelos que fueron ocurriendo, con la forma (número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado). En base al estado, podría contar con algún dato en blanco, por ejemplo si el vuelo fue cancelado no tendrá información de fechas y horas, si el vuelo se encuentra aún en curso, no contendrá información de la llegada. Se pide resolver utilizando PySpark:

A) Cuál es el aeropuerto con mayor tránsito.
B) Cuál es la aerolínea con mayor cantidad de vuelos.
C) Cuál es la aerolínea con mayor cantidad de cancelaciones.
D) Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de salida.
E) Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de llegada.
F) Cuál es la aerolínea más puntual.
G) Cuál es el aeropuerto que registra mayor desviación con respecto a los horarios coordinados.

In [83]:
vuelos =  [(1, 2, 'AEP', 'MDQ', '20200320', '06:15', '20200320', '07:05'),
 (2, 1, 'AEP', 'CPC', '20200320', '12:00', '20200320', '14:15'),
 (3, 3, 'BRC', 'EZE', '20200320', '17:15', '20200320', '19:30'),
 (4, 3, 'AEP', 'IGR', '20200320', '21:00', '20200320', '22:40'),
 (5, 3, 'MDQ', 'IGR', '20200321', '06:00', '20200321', '08:00'),
 (6, 1, 'IGR', 'USH', '20200321', '21:00', '20200322', '04:20'),
 (7, 3, 'USH', 'AEP', '20200322', '15:15', '20200322', '18:20'),
 (8, 2, 'AEP', 'RGA', '20200322', '17:00', '20200322', '20:15'),
 (9, 3, 'BHI', 'COR', '20200323', '09:10', '20200323', '14:00'),
 (10, 2, 'NQN', 'BRC', '20200323', '14:45', '20200323', '15:05')]


estados =  [(1, 'LATAM', '20200320', '06:15', '20200320', '07:05', 'Finalizado'),
 (2, 'AMERICAN', '20200320', '12:00', '20200320', '14:28', 'Finalizado'),
 (3, 'IBERIA', '20200320', '17:15', None, None, 'En curso'),
 (4, 'IBERIA', None, None, None, None, 'Cancelado'),
 (5, 'IBERIA', '20200321', '06:23', '20200321', '08:30', 'Finalizado'),
 (6, 'AMERICAN', None, None, None, None, 'Cancelado'),
 (7, 'IBERIA', '20200322', '15:15', '20200322', '18:20', 'Finalizado'),
 (8, 'LATAM', '20200322', '17:05', None, None, 'En curso'),
 (9, 'IBERIA', None, None, None, None, 'Cancelado'),
 (10, 'LATAM', '20200323', '14:45', '20200323', '15:05', 'Finalizado')]

In [84]:
vuelos = sc.parallelize(vuelos)
estados = sc.parallelize(estados)

A) Cuál es el aeropuerto con mayor tránsito.

RDD = (número de vuelo, código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM)

rdd2 =  (número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado)

In [85]:
vuelo_y_estado = estados.map(lambda x: (x[0],x[6])).cache()
vuelos_no_cancelados = vuelo_y_estado.filter(lambda x: (x[1] != 'Cancelado'))
vuelos_no_cancelados.collect()


[(1, 'Finalizado'),
 (2, 'Finalizado'),
 (3, 'En curso'),
 (5, 'Finalizado'),
 (7, 'Finalizado'),
 (8, 'En curso'),
 (10, 'Finalizado')]

In [86]:
aeropuertos_transitados = vuelos.map(lambda x: (x[0],(x[2],x[3])))
aeropuertos_transitados.collect()

[(1, ('AEP', 'MDQ')),
 (2, ('AEP', 'CPC')),
 (3, ('BRC', 'EZE')),
 (4, ('AEP', 'IGR')),
 (5, ('MDQ', 'IGR')),
 (6, ('IGR', 'USH')),
 (7, ('USH', 'AEP')),
 (8, ('AEP', 'RGA')),
 (9, ('BHI', 'COR')),
 (10, ('NQN', 'BRC'))]

In [87]:
vuelos_transitados = vuelos_no_cancelados.leftOuterJoin(aeropuertos_transitados)
vuelos_transitados.collect()

[(8, ('En curso', ('AEP', 'RGA'))),
 (1, ('Finalizado', ('AEP', 'MDQ'))),
 (5, ('Finalizado', ('MDQ', 'IGR'))),
 (2, ('Finalizado', ('AEP', 'CPC'))),
 (10, ('Finalizado', ('NQN', 'BRC'))),
 (3, ('En curso', ('BRC', 'EZE'))),
 (7, ('Finalizado', ('USH', 'AEP')))]

In [88]:
aeropuerto_salida = vuelos_transitados.map(lambda x: (x[1][1][0],1)).cache()
aeropuerto_llegada = vuelos_transitados.map(lambda x: (x[1][1][1],1)).cache()
print(aeropuerto_llegada.take(5))
print(aeropuerto_salida.take(5))

[('RGA', 1), ('MDQ', 1), ('IGR', 1), ('CPC', 1), ('BRC', 1)]
[('AEP', 1), ('AEP', 1), ('MDQ', 1), ('AEP', 1), ('NQN', 1)]


In [89]:
aeropuerto_llegada = aeropuerto_llegada.reduceByKey(lambda x,y: x + y).cache()
aeropuerto_salida = aeropuerto_salida.reduceByKey(lambda x,y: x + y).cache()
print(aeropuerto_llegada.take(5))
print(aeropuerto_salida.take(5))

[('MDQ', 1), ('AEP', 1), ('RGA', 1), ('IGR', 1), ('CPC', 1)]
[('AEP', 3), ('MDQ', 1), ('NQN', 1), ('BRC', 1), ('USH', 1)]


In [92]:
aeropuerto_llegada.union(aeropuerto_salida).reduceByKey(lambda x,y: x+y).reduce(lambda x,y: x if x[1] > y[1] else y)[0]

'AEP'

B) Cuál es la aerolínea con mayor cantidad de vuelos.

In [91]:
estados.filter(lambda x: x[6] != 'Cancelado').map(lambda x: (x[1],1))\
.reduceByKey(lambda x,y : x+y)\
.reduce(lambda x,y: x if x[1] > y[1] else y)[0]

'IBERIA'

C) Cuál es la aerolínea con mayor cantidad de cancelaciones.